## Load modules and packages

In [ ]:
import numpy as np
import pandas as pd
import datetime
from tqdm.notebook import tqdm
from random import sample
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn import tree
from sklearn.metrics import roc_auc_score
import random
from sklearn.pipeline import Pipeline
import category_encoders as ce
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.experimental import enable_halving_search_cv # noqa
# now you can import normally from model_selection
from sklearn.model_selection import HalvingRandomSearchCV

## Load data

In [ ]:
random.seed(10) 

from functions import *
# Load data - set index column, decimal point, separator
data = pd.read_csv('hw1_devsample.csv', sep=',',
                   decimal='.', index_col='SK_ID_CURR')

# print time of data being loaded - use strftime
print(f'Data loaded on:   {datetime.datetime.now().strftime(format="%Y-%m-%d %H:%M:%S")}')
data_cart = data.copy()

data_test = pd.read_csv('hw1_outofsample.csv', sep=',',decimal='.', index_col='SK_ID_CURR')

## Small changes

In [ ]:
target = data_cart['TARGET']
data_cart = data_cart.drop(['MONTH','TIME','DAY','BASE','TARGET'], axis=1)
data_cart.info()

In [ ]:
cols_pred = data_cart.columns
cols_pred_cat = [col for col in cols_pred if data_cart[col].dtype == 'O']
cols_pred_cat


## Split data train and test

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_cart2, target, test_size=0.2, random_state=123)

# Test target encoder

In [ ]:
target_encoder = ce.target_encoder.TargetEncoder(verbose=0, 
cols=cols_pred_cat,
#drop_invariant=False, 
return_df=True, handle_missing='value', 
handle_unknown='value', min_samples_leaf=1
#, smoothing=1.0
)

In [ ]:
data_cart2 = target_encoder.fit_transform(data_cart,target)
data_cart2.info()

In [ ]:
data_cart2[data_cart2.columns].isna().sum()

In [ ]:
# find columns with infinity values
cols_with_inf = []
for col in data_cart2.columns:
    if np.any(np.isinf(data_cart2[col])):
        cols_with_inf.append(col)
        print(f'Column {col} includes infinity values.')

# find columns with negative infinity values
cols_with_neginf = []
for col in data_cart2.columns:
    if np.any(np.isneginf(data_cart2[col])):
        cols_with_neginf.append(col)
        print(f'Column {col} includes negative infinity values.')

for col in cols_with_inf:
    data_cart2[col].replace(np.inf, 9999999, inplace = True)

# Test XGBOOST

In [ ]:
xgb_test = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='multi:softprob', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

In [ ]:
xgb_guillaume = xgb.XGBClassifier()

In [ ]:
xgb_test.fit(X_train,y_train)

# Pipeline

In [ ]:
pipe = Pipeline([
    ('Mean Target Encoding', target_encoder),
    ('model', xgb_test)
])

## Halving Random Search CV

In [ ]:
param_grid = {
    'select__k': [5, 10, 15],
    'model__max_depth': [2, 4, 6, 8]
}

## Apply pipeline to data

In [ ]:
search = HalvingRandomSearchCV(pipe, param_grid, cv=3).fit(X_train, y_train)

## Calculate AUC

# Predict on the second dataset

In [ ]:
prediction = ...

In [ ]:
data_saving = pd.DataFrame({'SK_ID_CURR' : data_test.index ,'prediction' : prediction})
data_saving.to_csv('DS2_22_HW1_CADIOU&PAIN.csv',index=False,sep=',')
data_saving.info()